In [272]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from bs4 import BeautifulSoup
from datetime import date, timedelta, datetime
import re
import pandas as pd
import os


def Getting_Flight_Info(URL):
    driver = webdriver.Chrome('/Users/robjohns/Documents/Metis/Project2/chromedriver')
    driver.get(URL)
    time.sleep(2)
    soup = BeautifulSoup(driver.page_source, 'lxml')
    driver.quit()
    data=DataGetter(URL,soup)
    return data



In [241]:
print(super_short_test_list_dates)
print(super_short_test_list)

['2019-07-22', '2019-09-03', '2019-10-16', '2019-11-28', '2019-07-22', '2019-09-03', '2019-10-16', '2019-11-28']
['https://www.google.com/flights?hl=en#flt=SEA.SFO.2019-07-22;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o', 'https://www.google.com/flights?hl=en#flt=SEA.SFO.2019-09-03;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o', 'https://www.google.com/flights?hl=en#flt=SEA.SFO.2019-10-16;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o', 'https://www.google.com/flights?hl=en#flt=SEA.SFO.2019-11-28;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o', 'https://www.google.com/flights?hl=en#flt=SEA.ORD.2019-07-22;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o', 'https://www.google.com/flights?hl=en#flt=SEA.ORD.2019-09-03;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o', 'https://www.google.com/flights?hl=en#flt=SEA.ORD.2019-10-16;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o', 'https://www.google.com/flights?hl=en#flt=SEA.ORD.2019-11-28;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o']


In [273]:


for URL in full_url_list[978:]:
    data=Getting_Flight_Info(URL)
    data=pd.DataFrame(data)
    
    Locations = re.search('#flt=(.+?);b:1', URL).group(1)
    Spots=Locations.split('.')
    Departure=Spots[0]
    Arrival = Spots[1]
    Date= Spots[2]
    Date=Date.replace('-',"_")
     
    data.to_pickle(f'Picklefolder/{Departure}_{Arrival}_{Date}')
    print(f'did {name}')

#testy=pd.read_pickle('Picklefolder/testpickle')
#print(testy)
     

num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20



num of flights: 20

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 20

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 18

did 2019-11-28
num of flights: 19

did 2019-11-28
num of flights: 18

did 2019-11-28
num of flights: 18



num of flights: 5

did 2019-11-28
num of flights: 5

did 2019-11-28
num of flights: 5

did 2019-11-28
num of flights: 5

did 2019-11-28
num of flights: 5

did 2019-11-28
num of flights: 6

did 2019-11-28
num of flights: 6

did 2019-11-28
num of flights: 6

did 2019-11-28
num of flights: 5

did 2019-11-28
num of flights: 5

did 2019-11-28
num of flights: 6

did 2019-11-28
num of flights: 6

did 2019-11-28
num of flights: 6

did 2019-11-28
num of flights: 6

did 2019-11-28
num of flights: 6

did 2019-11-28
num of flights: 5

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights: 7

did 2019-11-28
num of flights

num of flights: 8

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 9

did 2019-11-28
num of flights: 9

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 8

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 9

did 2019-11-28
num of flights: 9

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 8

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 9

did 2019-11-28
num of flights: 9

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 8

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 9

did 2019-11-28
num of flights: 9

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 10

did 2019-11-28
num of flights: 8

did 2019-11-2

num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 12

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 12

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 13

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 12

did 2019-11-28
num of flights: 12

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 14

did 2019-11-28
num of flights: 12

did 2019-11-28


In [255]:
filelist = os.listdir('Picklefolder/') 

In [256]:
filelist


['SEA_SFO_2019_09_03',
 'SEA_ORD_2019_10_16',
 '.DS_Store',
 'SEA_ORD_2019_11_28',
 'SEA_ORD_2019_07_22',
 'SEA_SFO_2019_07_22',
 'SEA_SFO_2019_10_16',
 'SEA_ORD_2019_09_03',
 'SEA_SFO_2019_11_28']

In [329]:

#testy=pd.read_pickle('Picklefolder/testpickle')

#list the files
filelist = os.listdir('Picklefolder/') 
#read them into pandas
df_list = [pd.read_pickle(f'Picklefolder/{file}') for file in filelist if file[0]!='.']
#concatenate them together
df = pd.concat(df_list,ignore_index=True)
df.tail(5)
#print(testy)



,0,1,2,3,4,5,6,7,8,9,10
21468,https://www.google.com/flights?hl=en#flt=SEA.O...,2019-10-31,SEA,ORD,3.83,160,11:30 P,5:20 A,American,AA1408,Boeing 737
21469,https://www.google.com/flights?hl=en#flt=SEA.O...,2019-10-31,SEA,ORD,3.90,175,7:00 A,12:54 P,United,UA2287,Boeing 737
21470,https://www.google.com/flights?hl=en#flt=SEA.O...,2019-10-31,SEA,ORD,3.97,175,11:04 A,5:02 P,United,UA278,Boeing 737
21471,https://www.google.com/flights?hl=en#flt=SEA.O...,2019-10-31,SEA,ORD,3.85,175,2:10 P,8:01 P,United,UA670,Boeing 737
21472,https://www.google.com/flights?hl=en#flt=SEA.O...,2019-10-31,SEA,ORD,3.78,182,5:55 P,11:42 P,Delta,DL853,Airbus A319


In [330]:
df.dtypes
name_dict={0:'URL',1:'Date',2:'Origin',3:'Destination',4:'FlightDuration',5:'Price',6:'Time_depart',
          7:'Time_arrive',8:'Airline',9:'FlightNumber',10:'PlaneType'}
df.rename(name_dict,axis=1,inplace=True)

In [331]:
df.head(100)

,URL,Date,Origin,Destination,FlightDuration,Price,Time_depart,Time_arrive,Airline,FlightNumber,PlaneType
0,https://www.google.com/flights?hl=en#flt=SEA.B...,2019-09-12,SEA,BOS,5.17,124,3:25 P,11:35 P,Delta,DL341,Boeing 737
1,https://www.google.com/flights?hl=en#flt=SEA.B...,2019-09-12,SEA,BOS,5.18,179,11:59 P,8:10 A,JetBlue,"(REGEXerror:, Airbus A320B6 498)","(REGEXerror:, Airbus A320B6 498)"
2,https://www.google.com/flights?hl=en#flt=SEA.B...,2019-09-12,SEA,BOS,5.33,184,3:50 P,12:10 A,Alaska,AS736,Boeing 737
3,https://www.google.com/flights?hl=en#flt=SEA.B...,2019-09-12,SEA,BOS,5.50,164,7:50 A,4:20 P,Delta,DL1163,Boeing 737
4,https://www.google.com/flights?hl=en#flt=SEA.B...,2019-09-12,SEA,BOS,5.37,179,11:33 A,7:55 P,JetBlue,"(REGEXerror:, Airbus A320B6 598)","(REGEXerror:, Airbus A320B6 598)"
5,https://www.google.com/flights?hl=en#flt=SEA.B...,2019-09-12,SEA,BOS,5.33,184,9:15 P,5:35 A,Alaska,AS24,Boeing 737
6,https://www.google.com/flights?hl=en#flt=SEA.B...,2019-09-12,SEA,BOS,5.35,204,10:40 P,7:01 A,Delta,DL2635,Boeing 737
7,https://www.google.com/flights?hl=en#flt=SEA.B...,2019-09-12,SEA,BOS,5.50,339,6:50 A,3:20 P,Alaska,AS12,Boeing 737
8,https://www.google.com/flights?hl=en#flt=SEA.J...,2019-08-09,SEA,JFK,5.70,260,1:18 P,10:00 P,JetBlue,"(REGEXerror:, Airbus A320B6 464)","(REGEXerror:, Airbus A320B6 464)"
9,https://www.google.com/flights?hl=en#flt=SEA.J...,2019-08-09,SEA,JFK,5.43,260,10:26 P,6:52 A,JetBlue,"(REGEXerror:, Airbus A320B6 264)","(REGEXerror:, Airbus A320B6 264)"


In [332]:
#Cleaning up Data types

df["Price"] = pd.to_numeric(df["Price"], errors='coerce')
df["Date"] = pd.to_datetime(df["Date"])


df["Time_depart"] = df["Time_depart"].apply(lambda x: f'{x}M')
df["Time_arrive"] = df["Time_arrive"].apply(lambda x: f'{x}M')

df["Time_depart"] = pd.to_datetime(df["Time_depart"],format='%I:%M %p',errors='coerce')
df["Time_arrive"] = pd.to_datetime(df["Time_arrive"],format='%I:%M %p',errors='coerce')

df.dtypes

URL                       object
Date              datetime64[ns]
Origin                    object
Destination               object
FlightDuration           float64
Price                    float64
Time_depart       datetime64[ns]
Time_arrive       datetime64[ns]
Airline                   object
FlightNumber              object
PlaneType                 object
dtype: object

In [366]:
df2=df
df2=df2.dropna(axis=0)
#df.describe()
df2.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 20565 entries, 0 to 21472
Data columns (total 11 columns):
URL               20565 non-null object
Date              20565 non-null datetime64[ns]
Origin            20565 non-null object
Destination       20565 non-null object
FlightDuration    20565 non-null float64
Price             20565 non-null float64
Time_depart       20565 non-null datetime64[ns]
Time_arrive       20565 non-null datetime64[ns]
Airline           20565 non-null object
FlightNumber      20565 non-null object
PlaneType         20565 non-null object
dtypes: datetime64[ns](3), float64(2), object(6)
memory usage: 1.9+ MB


In [382]:
#df3=df2.groupby(['Destination','Date']).agg({'Price':min})
#df2=df2.sort_values('Price')
df3=(df2.sort_values(['Price'], ascending=[True]).groupby(['Destination','Date'])
 .head(1))

In [399]:
df3.sort_values(['Destination','Date'])

basedate = pd.Timestamp('2019-07-22')
df3['Days'] = df3['Date'].apply(lambda x: (x - basedate).days)
df3.info()
df3.tail(3)


<class 'pandas.core.frame.DataFrame'>
Int64Index: 1772 entries, 1691 to 15136
Data columns (total 12 columns):
URL               1772 non-null object
Date              1772 non-null datetime64[ns]
Origin            1772 non-null object
Destination       1772 non-null object
FlightDuration    1772 non-null float64
Price             1772 non-null float64
Time_depart       1772 non-null datetime64[ns]
Time_arrive       1772 non-null datetime64[ns]
Airline           1772 non-null object
FlightNumber      1772 non-null object
PlaneType         1772 non-null object
Days              1772 non-null int64
dtypes: datetime64[ns](3), float64(2), int64(1), object(6)
memory usage: 180.0+ KB


,URL,Date,Origin,Destination,FlightDuration,Price,Time_depart,Time_arrive,Airline,FlightNumber,PlaneType,Days
4302,https://www.google.com/flights?hl=en#flt=SEA.A...,2019-10-31,SEA,AUS,4.17,566.0,1900-01-01 07:45:00,1900-01-01 13:55:00,Alaska,AS166,Boeing 737,101
13855,https://www.google.com/flights?hl=en#flt=SEA.A...,2019-07-22,SEA,ATL,4.83,634.0,1900-01-01 21:20:00,1900-01-01 05:10:00,Alaska,AS750,Boeing 737,0
15136,https://www.google.com/flights?hl=en#flt=SEA.A...,2019-07-23,SEA,ATL,4.92,634.0,1900-01-01 06:15:00,1900-01-01 14:10:00,Alaska,AS746,Boeing 737,1


In [232]:
##this will take the soupy html file and pull out data values for all flight results from a search##
# looking for [Date,DepartureCity,ArrivalCity,DepartureTime,
#               ArrivalTime,FlightDuration,Price,PlaneType,Airline,FlightNumber]
import re
#'https://www.google.com/flights?hl=en#flt=SFO.AUS.2019-08-21;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
#each page has a different number of flights on it, so for this URL we need to make a list of lists
#that is the length = number of flights on the page#


def DataGetter(URL,soup):

    Locations = re.search('#flt=(.+?);b:1', URL).group(1)
    Spots=Locations.split('.')
    Departure=Spots[0]
    Arrival = Spots[1]
    Date= Spots[2]



    flights=len(soup.find_all('div', attrs={'class': 'gws-flights-results__itinerary-duration'}))
    print('num of flights:',flights)
    BigList = [[] for _ in range(flights)]
    for flight in BigList:
        flight.append(URL)
        flight.append(Date)
        flight.append(Departure)
        flight.append(Arrival)

    # Append flight durations onto each element of the list
    i=-1
    for li in soup.find_all('div', attrs={'class': 'gws-flights-results__duration flt-subhead1Normal'}):
        i+=1
        chonk=li.get_text()
        try:
            ch=chonk.split('h')
            onk=chonk.split(' ')
            num2=onk[1]
            flight_duration=float(ch[0])+(float(num2[:-1])/60)
            flight_duration = round(flight_duration,2)
            BigList[i].append(flight_duration)
            #print ('flight_duration:',flight_duration)
        except:
            BigList[i].append('NaN')
            print('There was an issue')



    #for li in soup.find_all('div', attrs={'class': 'gws-flights-results__price'}):
        #print(li)


    #here we add more flight info from the jstcache. . . beware this number seems to change from day to day
    i=-1
    for li in soup.find_all('jsl', attrs={'jstcache': "8836"}):
        i+=1
        text = li.get_text()

          #finding price
        try:
            found = re.search('From (.+?)Trip', text).group(1)
            dollar=found.replace(',','')
            found=float(dollar[1:-1])
        except AttributeError:
        # if the sentence structure i didnt expect show up
            found = 'NaN' # apply your error handling
        #print('cost:',found)
        BigList[i].append(found)
        #finding departure and arrival times
        try:
            found = re.search('Departure time: (.+?)M', text).group(1)
            found2 = re.search('Arrival time: (.+?)M', text).group(1)

            #departure=found.replace(',','')
            #found=float(dollar[1:-1])
        except AttributeError:
        # if the sentence structure i didnt expect show up
            found = 'NaN' # apply your error handling
        #print('Departure Time:',found)
        #print('Arrival Time:',found2)
        BigList[i].append(found)
        BigList[i].append(found2)


        #grab airline
        try:
            found = re.search('by (.+?).Depa', text).group(1)

            #departure=found.replace(',','')
            #found=float(dollar[1:-1])
        except AttributeError:
        # if the sentence structure i didnt expect show up
            found = 'NaN' # apply your error handling
        #print('Airline:',found)
        BigList[i].append(found)
    #print('')
        #grab PlaneType and flight number
    i=-1    
    for li in soup.find_all('div', attrs={'class': 'gws-flights-results__other-leg-info'}):
            i+=1
            #print(li.get_text())
            st=li.get_text()
            #st=str(st.text())
            result = re.match("(?P<PlaneType>[A-Za-z]+.[A-Za-z]*[0-9]+)(?P<FlightNum>[A-Za-z]+\s[0-9]+)", st)
            if result:
                Planetype=result.group('PlaneType')
                Flightnumber=result.group('FlightNum')
                Flightnumber=Flightnumber.replace('\xa0','')
                #print('Flightnum:',Flightnumber)
                #print('Planetype:',Planetype)
                BigList[i].append(Flightnumber)
                BigList[i].append(Planetype)
                #print("")
            else:
                errormsg=('REGEXerror:',st)
                BigList[i].append(errormsg)
                BigList[i].append(errormsg)

    return BigList

In [231]:
for lists in BigList:
    print(len(lists))

11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11
11


In [275]:
##Format URLs, can create these lists:      super_short_test_list//test_urls//full_cities_lessdates_urls//
##//full_dates_lesscities_urls//full_url_list//come_home_URLS##

cities=['SFO','AUS','ATL','BOI','BOS','ORD','DEN','DTW','FAI','JFK','SAN']

##links look like this for one ways,nonstop, with overhead bags allowed, and frontier airlines excluded##
'https://www.google.com/flights?hl=en#flt=SFO.AUS.2019-08-21;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'

#This makes all the dates from july 22nd to the end of the year, will update from date time later if i have time
dates=['2019-07-22']
for days in range(23,32):
    date = '2019-07-' + str(days)
    dates.append(date)
for days in range(1,32):
    if days>=10:
        date = '2019-08-' + str(days)
        dates.append(date)
    else:
        date = '2019-08-0' + str(days)
        dates.append(date)
for days in range(1,31):
    if days>=10:
        date = '2019-09-' + str(days)
        dates.append(date)
    else:
        date = '2019-09-0' + str(days)
        dates.append(date)
for days in range(1,32):
    if days>=10:
        date = '2019-10-' + str(days)
        dates.append(date)
    else:
        date = '2019-10-0' + str(days)
        dates.append(date) 

for days in range(1,31):
    if days>=10:
        date = '2019-11-' + str(days)
        dates.append(date)
    else:
        date = '2019-11-0' + str(days)
        dates.append(date)           

for days in range(1,32):
    if days>=10:
        date = '2019-12-' + str(days)
        dates.append(date)
    else:
        date = '2019-12-0' + str(days)
        dates.append(date)         

## This makes lists of URLS to use generating data##
super_short_test_list=[]
super_short_test_list_dates=[]
for city in cities[::6]:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates[::43]:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        super_short_test_list.append(URL)
        super_short_test_list_dates.append(date)
        
print('super short test set has this many values:'+str(len(super_short_test_list)))

test_urls=[]
for city in cities[::5]:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates[::29]:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        test_urls.append(URL)
print('test set has this many values:'+str(len(test_urls)))        
        
full_cities_lessdates_urls=[]
for city in cities:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates[::15]:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        full_cities_lessdates_urls.append(URL)
print('full city set has this many values:'+str(len(full_cities_lessdates_urls)))   

full_dates_lesscities_urls=[]
for city in cities[::3]:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        full_dates_lesscities_urls.append(URL)
print('full date set has this many values:'+str(len(full_dates_lesscities_urls)))  

full_url_list=[]
for city in cities:
    URLc='https://www.google.com/flights?hl=en#flt=SEA.'+city+'.'
    
    for date in dates:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        full_url_list.append(URL)
print('full set has this many values:'+str(len(full_url_list)))  

come_home_URLS=[]
for city in cities:
    URLc='https://www.google.com/flights?hl=en#flt='+city+'.'+'SEA.'
    for date in dates:
        URL=URLc+date+';b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o'
        come_home_URLS.append(URL)
print('come home set has this many values:'+str(len(come_home_URLS)))  



super short test set has this many values:8
test set has this many values:18
full city set has this many values:121
full date set has this many values:652
full set has this many values:1793
come home set has this many values:1793


https://www.google.com/flights?hl=en#flt=SEA.ORD.2019-07-22;b:1;c:USD;e:1;s:0;a:-F9;sd:1;t:f;tt:o


In [ ]:
## how to avoid booting from server ideas . . . .
for page in super_short_test_list:
    ### scrape a website
    ### ...
    print(page)
    
    time.sleep(2)
    #time.sleep(.5+2*random.random())
    
import sys
import requests
from bs4 import BeautifulSoup

url = 'http://www.reddit.com'

user_agent = {'User-agent': 'Mozilla/5.0'}
response  = requests.get(url, headers = user_agent)

from fake_useragent import UserAgent

ua = UserAgent()
user_agent = {'User-agent': ua.random}
print(user_agent)

response  = requests.get(url, headers = user_agent)
print(response.text)